<a href="https://colab.research.google.com/github/davidraamirez/GradientWithoutBackpropagation/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gradient Without Backpropagation

In [23]:
import torch
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
import tqdm
import torch.distributions as distr

In [24]:
%pip install torchmetrics --quiet

In [25]:
import torchmetrics
import torchvision
from torchvision import transforms as T

Loading and preprocessing the dataset

In [26]:
#Load the dataset
train_data = torchvision.datasets.KMNIST('./data', train=True, download=True)

In [27]:
# Alternative way to get the first element: image, label = next(iter(train_data))
for image, label in train_data:
  break

In [28]:
# Simple transformation that converts the PIL image to a PyTorch array
T.ToTensor()(image).shape

torch.Size([1, 28, 28])

In [29]:
# T.Compose allows to chain together multiple transformations
train_transforms = T.Compose([
    T.ToTensor()
])

In [30]:
# This loads data with both data conversion.
train_data = torchvision.datasets.KMNIST('./data', train=True, transform=train_transforms)

In [31]:
# Loaders are used to shuffle, batch, and possibly sample the elements of the dataset
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

In [32]:
xb, yb = next(iter(train_loader))
print(xb.shape)
print(yb.shape)

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [33]:
# Loading the test data is similar, but (a) we do not apply data augmentation,
# and (b) we do not shuffle when building the mini-batches.
test_data = torchvision.datasets.KMNIST('./data', train=False, transform=T.ToTensor())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

Initialize parameters ????!!!!!

In [34]:
#Initialize parameters

def init():
  theta = torch.randn(X.shape, requires_grad=False)
  v = torch.randn(X.shape, requires_grad=False)
  return theta, v

Define Convolutional Neural Network Class

In [35]:
from torch import nn
from torch.nn import functional as F

In [56]:
class SimpleCNN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        input_size = 1
        self.conv1 = nn.Conv2d(input_size, 8, 3, padding=1)
        print(self.conv1.weight)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 64, 3, padding=1)
        self.max_pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64*7*7, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.max_pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.max_pool(x)
        x = x.reshape((-1, 64*7*7))
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [37]:
# We check if CUDA is available. If you do not see it,
# activate a GPU from Runtime >> Change runtime type and 
# restart the notebook.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [57]:
# We need to move the SimpleCNN model to the device immediately
cnn = SimpleCNN(1).to(device)
print(len(list(cnn.parameters())))

Parameter containing:
tensor([[[[ 0.0700, -0.0318, -0.3248],
          [-0.2118, -0.3014,  0.2572],
          [ 0.0616,  0.2601,  0.2349]]],


        [[[ 0.0264, -0.1425, -0.3194],
          [ 0.2587,  0.1923, -0.0372],
          [ 0.2086,  0.0800, -0.1155]]],


        [[[-0.3106, -0.3326,  0.0613],
          [ 0.3019,  0.0629,  0.1640],
          [-0.2718, -0.3292,  0.3142]]],


        [[[-0.0018,  0.1631,  0.3232],
          [ 0.1476, -0.2349, -0.0070],
          [-0.0350,  0.0577,  0.0889]]],


        [[[ 0.0672, -0.2210,  0.1980],
          [-0.0196, -0.0907, -0.0714],
          [-0.0437,  0.1285, -0.2404]]],


        [[[-0.2319,  0.2314,  0.2135],
          [ 0.0706, -0.3154, -0.0576],
          [-0.0807,  0.2794, -0.2058]]],


        [[[ 0.1104, -0.1419,  0.2641],
          [ 0.0340,  0.1222, -0.0401],
          [-0.0549,  0.2739,  0.2505]]],


        [[[-0.1748, -0.1398,  0.1064],
          [-0.0299,  0.0014,  0.2878],
          [-0.1388, -0.0686, -0.2917]]]], requires_gr

In [41]:
# Note: we also need to move data when asking for a prediction
cnn(xb.to(device)).shape

torch.Size([64, 10])

Train and evaluate the network with forward gradient

In [42]:
def accuracy(net, loader, device):
  # A function that aggregates the accuracy over all mini-batches in the loader.
  # See here for a quick-start on torchmetrics: https://torchmetrics.readthedocs.io/en/stable/pages/quickstart.html.
  #acc = torchmetrics.Accuracy().to(device)
  acc = torchmetrics.Accuracy('multiclass', num_classes=10).to(device)
  for xb, yb in loader:
      xb, yb = xb.to(device), yb.to(device)
      ypred = cnn(xb)
      _ = acc(ypred, yb)
  return acc.compute()

In [43]:
# Average accuracy at initialization is 10% (random guessing).
accuracy(cnn, test_loader, device)

tensor(0.0969, device='cuda:0')

DEFINE CROSS_ENTROPY

In [44]:
# Note: it is important to move the CNN to the device before initializing the optimizer,
# since the optimizer also has a state that must be moved to the GPU.
loss = nn.CrossEntropyLoss()


In [45]:
def beale_function(x):
  return (torch.pow(torch.tensor([1.5])-x[0]+x[0]*x[1],2) + torch.pow(torch.tensor([2.25])-x[0]+x[0]*torch.pow(x[1],2),2)+torch.pow(torch.tensor([2.625])-x[0]+x[0]*torch.pow(x[1],3),2))

In [46]:
def train_fp(x, y):
  x, y = x.to(device), y.to(device)

  l_rate0 = 0.025
  f = beale_function
  theta1 = torch.rand(2)
  theta0 = theta1 + torch.tensor([1,1])
  t=torch.tensor([0])
  while (torch.norm(theta1 - theta0)>1e-10) :
    l_rate=l_rate0*torch.exp(-t*1e-4)
    t=t+1
    v=torch.normal(torch.tensor([0.0, 0.0]),torch.tensor([1.0, 1.0]))
    ft=f(theta1)
    dt=torch.tensor(torch.autograd.functional.jvp(f,theta1,v)[1])
    gt=v*dt
    theta0 = theta1
    theta1 = theta1 - l_rate*gt

  return theta1  

In [47]:
x= torch.tensor([2, 4, 8, 16])
y=torch.tensor([1,2,3,4,5,6,7,8,9,10])
theta=train_fp(x,y)
print(theta)

<ipython-input-46-08ee1067b7d1>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dt=torch.tensor(torch.autograd.functional.jvp(f,theta1,v)[1])


tensor([3.0004, 0.5001])


In [55]:
for epoch in range(3):

  cnn.train()
  for i in range(10):
    xb, yb = next(iter(train_loader))
    xb = xb.to(device)
    yb = yb.to(device)
    ypred = cnn(xb)
    l = loss(ypred, yb)

    #Update cnn parameters
    #Recalculate ypred and loss
    #MIRAR NN_LAB_LOGISITC_REGRESSION
    #CALCULAR G(THETA) QUE ES EL GRADIENTE Y APLICARLO A LOS PARAMETROS DEL CNN, LOS WEIGHTS

  cnn.eval()
  print(f'Accuracy at epoch {epoch}: {accuracy(cnn, test_loader, device)}')

RuntimeError: ignored